In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt


In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
  Base = automap_base()
# reflect the tables
 Base.prepare(autoload_with=engine)

In [ ]:
# View all of the classes that automap found
 Base.classes.keys()

In [ ]:
# Save references to each table
 station = Base.classes.station
measurement = Base.classes.measurement

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
 recent_date = session.query(measurement.date).order_by(measurement.date.desc()).first()
recent_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
year_ago = dt.date(2017, 8, 23) - dt.timedelta(days=365)
year_ago

# Perform a query to retrieve the data and precipitation scores
data = session.query(measurement.date, measurement.prcp).\
    filter(measurement.date > (year_ago)).order_by(measurement.date.desc())

# Save the query results as a Pandas DataFrame. Explicitly set the column names
df = pd.DataFrame(data, columns=['date', 'precipitation'])
df = df.set_index("date")

# Sort the dataframe by date
df = df.sort_values(by=['date'])
df

# Use Pandas Plotting with Matplotlib to plot the data
df.plot()
plt.xticks(rotation='vertical', fontsize=10)
plt.xlabel("Date") 
plt.ylabel("Inches")
plt.tight_layout()
plt.show()

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
tot_stations = session.query(station).distinct().count()
tot_stations

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
sel = [station.station, func.count(measurement.station)]
active_query = session.query(*sel).filter(station.station == measurement.station).group_by(station.station).\
    order_by(func.count(measurement.station).desc()).all()

active_query

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
sel = [func.min(measurement.tobs),
      func.max(measurement.tobs),
      func.avg(measurement.tobs)]

active_data = session.query(*sel).filter(measurement.station == 'USC00519281').all()
active_data

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
last_year_data = session.query(measurement.date, measurement.tobs).filter(measurement.station == 'USC00519281', \
                                                        measurement.date >= year_ago).all()
df_last_year = pd.DataFrame(last_year_data, columns=['date', 'temperature'])
df_last_year

In [ ]:
# Create a histogram based off of the data
df_last_year.hist(bins=12)
plt.xlabel("Temperature") 
plt.ylabel("Frequency")
plt.title("Tobs")
plt.show()

# Close Session

In [ ]:
# Close Session
session.close()